In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import os
import timm
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Parameters
img_size = 64
batch_size = 64
epochs = 30
learning_rate = 1e-4
min_lr = 1e-6
weight_decay = 1e-4
patience = 3

# Paths
data_dir = '/kaggle/input/emotion'
save_path = '/kaggle/working/emotion_mv_improved.pth'

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Compute dataset mean and std
def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    mean = 0.
    std = 0.
    nb_samples = 0.
    
    for data, _ in loader:
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples
    
    mean /= nb_samples
    std /= nb_samples
    return mean.item(), std.item()

tmp_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

tmp_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=tmp_tf)
gray_mean, gray_std = compute_mean_std(tmp_ds)
print(f'Computed gray mean: {gray_mean:.4f}, std: {gray_std:.4f}')

# Data Augmentation
train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5)),
    transforms.ToTensor(),
    transforms.Normalize([gray_mean], [gray_std]),
])

test_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([gray_mean], [gray_std]),
])

# Datasets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_tf)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_tf)

# Get class information
classes = train_dataset.classes
num_classes = len(classes)
print('Classes:', classes, "Number of classes:", num_classes)

# Handle class imbalance
class_counts = np.bincount([label for _, label in train_dataset])
print("Class counts:", class_counts)
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = class_weights[[label for _, label in train_dataset]]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# Data Loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=sampler,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# CORRECTED MODEL CREATION - Simplified approach
model = timm.create_model(
    'mobilevit_s',
    pretrained=True,
    num_classes=num_classes,  # Set to your actual number of classes
    img_size=(img_size, img_size),
    in_chans=1,
    drop_rate=0.2,
    drop_path_rate=0.1
).to(device)

# Print model to verify output size
print(model)
print(f"Model output size: {model.num_classes} (should match {num_classes})")

# Loss function with class weights
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Scheduler
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=patience,
    min_lr=min_lr
)

# Training
best_acc = 0.0
best_epoch = 0
early_stop_counter = 0

for epoch in range(1, epochs + 1):
    model.train()
    running_loss = 0.0
    
    for imgs, labels in train_loader:
        # Move data to device
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in test_loader:
            # Move data to device
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    val_loss /= len(test_loader.dataset)
    test_acc = correct / total * 100
    
    # Update learning rate
    scheduler.step(test_acc)
    
    print(f"Epoch {epoch:02d}:")
    print(f"  Train Loss: {epoch_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val Acc: {test_acc:.2f}%")
    print(f"  Current LR: {optimizer.param_groups[0]['lr']:.2e}")
    
    if epoch == epochs or test_acc > best_acc:
        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds, target_names=classes))
    
    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch
        early_stop_counter = 0
        torch.save(model.state_dict(), save_path)
        print(f"  New best model saved with acc={best_acc:.2f}%")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience * 2:
            print(f"Early stopping at epoch {epoch}")
            break
    
    print("-" * 50)

print(f"\nBest Test Accuracy: {best_acc:.2f}% at epoch {best_epoch}")

# Load best model
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

# Prediction function
def predict(img_path, return_confidence=False):
    img = Image.open(img_path).convert('L')
    x = test_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
    idx = logits.argmax(dim=1).item()
    if return_confidence:
        return classes[idx], probs[0, idx].item()
    return classes[idx]

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import os
import timm
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

# Parameters
img_size = 96
batch_size = 128
epochs = 100
learning_rate = 5e-4
min_lr = 1e-6
weight_decay = 1e-4
patience = 10
warmup_epochs = 5
num_workers = 4

# Paths
data_dir = '/kaggle/input/emotion'
save_path = '/kaggle/working/mobilevit_emotion_best.pth'
history_path = '/kaggle/working/mobilevit_training_history.png'

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Compute dataset mean and std
def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    mean = 0.
    std = 0.
    nb_samples = 0.
    
    for data, _ in tqdm(loader, desc="Computing mean/std"):
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples
    
    mean /= nb_samples
    std /= nb_samples
    return mean.item(), std.item()

# Create temporary transforms
tmp_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

print("Loading dataset for mean/std calculation...")
tmp_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=tmp_tf)
gray_mean, gray_std = compute_mean_std(tmp_ds)
print(f'Computed gray mean: {gray_mean:.4f}, std: {gray_std:.4f}')

# Enhanced Data Augmentation
train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0), ratio=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 1.0)),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize([gray_mean], [gray_std]),
])

test_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([gray_mean], [gray_std]),
])

# Datasets
print("Creating datasets...")
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_tf)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_tf)

# Get class information
classes = train_dataset.classes
num_classes = len(classes)
print('Classes:', classes, "Number of classes:", num_classes)

# Handle class imbalance
class_counts = np.bincount([label for _, label in train_dataset])
print("Class counts:", class_counts)
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)
sample_weights = class_weights[[label for _, label in train_dataset]]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# Data Loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=sampler,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

# Enhanced MobileViT Model with automatic channel detection
class MobileViTEmotion(nn.Module):
    def __init__(self, num_classes, img_size=96):
        super().__init__()
        # Use MobileViT as backbone
        self.backbone = timm.create_model(
            'mobilevit_s',
            pretrained=True,
            features_only=True,
            in_chans=1
        )
        
        # Get actual channel dimensions from backbone
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, img_size, img_size)
            features = self.backbone(dummy_input)
            self.last_channel = features[-1].shape[1]
            self.feature_size = features[-1].shape[2]
        
        print(f"MobileViT feature dimensions: {self.last_channel} channels, {self.feature_size}x{self.feature_size}")
        
        # Feature refinement
        self.conv1 = nn.Conv2d(self.last_channel, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        # Get features from MobileViT
        features = self.backbone(x)
        x = features[-1]  # Use the deepest feature map
        
        # Feature refinement
        x = F.silu(self.conv1(x))
        x = F.silu(self.bn1(self.conv2(x)))
        x = F.silu(self.bn2(self.conv3(x)))
        
        # Attention mechanism
        att = self.attention(x)
        x = x * att
        
        return self.classifier(x)

model = MobileViTEmotion(num_classes=num_classes, img_size=img_size).to(device)
print(model)

# Advanced Loss Function
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss
            
        if self.reduction == 'mean':
            return focal_loss.mean()
        return focal_loss

# Apply class weights
class_weights = class_weights.to(device)
criterion = FocalLoss(alpha=class_weights, gamma=1.5)

# Optimizer with differential learning rates
backbone_params = []
head_params = []
for name, param in model.named_parameters():
    if 'backbone' in name:
        backbone_params.append(param)
    else:
        head_params.append(param)

optimizer = optim.AdamW(
    [
        {'params': backbone_params, 'lr': learning_rate/10},
        {'params': head_params, 'lr': learning_rate}
    ],
    weight_decay=weight_decay
)

# Combined learning rate schedulers
scheduler_cosine = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,
    T_mult=2,
    eta_min=min_lr
)
scheduler_plateau = ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=3
)

# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

# Training function
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    progress = tqdm(loader, desc="Training")
    
    for imgs, labels in progress:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        
        progress.set_postfix(loss=loss.item())
    
    return running_loss / len(loader.dataset)

# Validation function
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        progress = tqdm(loader, desc="Validation")
        for imgs, labels in progress:
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = correct / total * 100
    return running_loss / len(loader.dataset), accuracy, all_preds, all_labels

# Training loop
best_acc = 0.0
early_stop_counter = 0

print("Starting training...")
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")
    
    # Learning rate warmup
    if epoch <= warmup_epochs:
        lr_scale = min(1., float(epoch) / warmup_epochs)
        for pg in optimizer.param_groups:
            pg['lr'] = lr_scale * pg.get('initial_lr', learning_rate)
    
    # Train
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    history['train_loss'].append(train_loss)
    
    # Validate
    val_loss, val_acc, val_preds, val_labels = validate(model, test_loader, criterion, device)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    # Update learning rate
    if epoch > warmup_epochs:
        scheduler_cosine.step()
        scheduler_plateau.step(val_acc)
    
    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), save_path)
        print(f"New best model saved with accuracy: {val_acc:.2f}%")
        
        # Print classification report for best model
        print("\nClassification Report:")
        print(classification_report(val_labels, val_preds, target_names=classes))
    else:
        early_stop_counter += 1
        print(f"No improvement for {early_stop_counter}/{patience} epochs")
    
    # Early stopping
    if early_stop_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

print(f"\nBest Validation Accuracy: {best_acc:.2f}%")

# Plot training history
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Loss History')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(history['val_acc'], 'g-', label='Val Accuracy')
plt.title('Accuracy History')
plt.xlabel('Epochs')
plt.legend()

plt.tight_layout()
plt.savefig(history_path)
plt.show()

# Load best model
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

# Prediction function
def predict(img_path, return_confidence=False):
    img = Image.open(img_path).convert('L')
    x = test_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = F.softmax(logits, dim=1)
    idx = logits.argmax(dim=1).item()
    confidence = probs[0, idx].item()
    
    if return_confidence:
        return classes[idx], confidence
    return classes[idx]
    

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import os
import timm
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import classification_report
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

# Enhanced Parameters
img_size = 112  # Increased resolution for facial details
batch_size = 64  # Reduced for better gradient updates
epochs = 100  # More training time
learning_rate = 1e-3  # Higher initial learning rate
min_lr = 1e-6
weight_decay = 1e-5  # Reduced regularization
patience = 15
warmup_epochs = 10
num_workers = 4
mixup_alpha = 0.4  # Data augmentation

# Paths
data_dir = '/kaggle/input/emotion'
save_path = '/kaggle/working/mobilevit_emotion_best.pth'
history_path = '/kaggle/working/training_history.png'

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Compute dataset mean and std
def compute_mean_std(dataset):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    mean = 0.
    std = 0.
    nb_samples = 0.
    
    for data, _ in tqdm(loader, desc="Computing mean/std"):
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples
    
    mean /= nb_samples
    std /= nb_samples
    return mean.item(), std.item()

# Create temporary transforms
tmp_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

print("Loading dataset for mean/std calculation...")
tmp_ds = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=tmp_tf)
gray_mean, gray_std = compute_mean_std(tmp_ds)
print(f'Computed gray mean: {gray_mean:.4f}, std: {gray_std:.4f}')

# Enhanced Data Augmentation
train_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomResizedCrop(img_size, scale=(0.7, 1.0), ratio=(0.75, 1.33)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.4, contrast=0.4),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.7),
    transforms.ToTensor(),
    transforms.Normalize([gray_mean], [gray_std]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.15), ratio=(0.3, 3.3)),
])

test_tf = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([gray_mean], [gray_std]),
])

# Datasets
print("Creating datasets...")
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_tf)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=test_tf)

# Get class information
classes = train_dataset.classes
num_classes = len(classes)
print('Classes:', classes, "Number of classes:", num_classes)

# Handle class imbalance
class_counts = np.bincount([label for _, label in train_dataset])
print("Class counts:", class_counts)
max_count = max(class_counts)
class_weights = torch.tensor([max_count / count for count in class_counts], dtype=torch.float)
sample_weights = class_weights[[label for _, label in train_dataset]]
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# Data Loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    sampler=sampler,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

# Enhanced MobileViT Model with Residual Connections
class MobileViTEmotion(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Use MobileViT as backbone
        self.backbone = timm.create_model(
            'mobilevit_s',
            pretrained=True,
            features_only=True,
            in_chans=1
        )
        
        # Get feature dimensions
        dummy = torch.zeros(1, 1, img_size, img_size)
        features = self.backbone(dummy)
        self.feature_channels = [f.shape[1] for f in features]
        print("Feature channels:", self.feature_channels)
        
        # Feature refinement with residual connections
        self.conv1 = nn.Conv2d(self.feature_channels[-1], 256, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(256)
        
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.shortcut = nn.Sequential(
            nn.Conv2d(self.feature_channels[-1], 128, kernel_size=1),
            nn.BatchNorm2d(128)
        )
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=1),
            nn.BatchNorm2d(64),
            nn.SiLU(),
            nn.Conv2d(64, 1, kernel_size=1),
            nn.Sigmoid()
        )
        
        # Classifier with more capacity
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(128, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.SiLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x):
        # Get features from MobileViT
        features = self.backbone(x)
        x = features[-1]  # Use the deepest feature map
        
        # Initial transformation
        x1 = F.silu(self.bn1(self.conv1(x)))
        
        # Residual block
        identity = self.shortcut(x)
        x2 = F.silu(self.bn2(self.conv2(x1)))
        x3 = self.bn3(self.conv3(x2))
        x3 += identity  # Residual connection
        x = F.silu(x3)
        
        # Attention mechanism
        att = self.attention(x)
        x = x * att
        
        return self.classifier(x)

model = MobileViTEmotion(num_classes=num_classes).to(device)
print(model)
print(f"Total parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

# Advanced Loss Function with Label Smoothing
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, smoothing=0.1, reduction='mean'):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smoothing = smoothing
        self.reduction = reduction
        
    def forward(self, inputs, targets):
        log_probs = F.log_softmax(inputs, dim=-1)
        nll_loss = -log_probs.gather(dim=-1, index=targets.unsqueeze(1)).squeeze(1)
        
        # Label smoothing
        smooth_loss = -log_probs.mean(dim=-1)
        nll_loss = (1 - self.smoothing) * nll_loss + self.smoothing * smooth_loss
        
        # Focal loss
        pt = torch.exp(-nll_loss)
        focal_loss = ((1 - pt) ** self.gamma) * nll_loss
        
        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss
            
        if self.reduction == 'mean':
            return focal_loss.mean()
        return focal_loss

# Apply class weights
class_weights = class_weights.to(device)
criterion = FocalLoss(alpha=class_weights, gamma=2.0, smoothing=0.1)

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Learning rate schedulers
scheduler_cosine = CosineAnnealingLR(
    optimizer,
    T_max=epochs - warmup_epochs,
    eta_min=min_lr
)
scheduler_plateau = ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=5
)

# Training history
history = {
    'train_loss': [],
    'val_loss': [],
    'val_acc': [],
    'lr': []
}

# Mixup augmentation
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
        
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Training function with mixup
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    progress = tqdm(loader, desc="Training")
    
    for imgs, labels in progress:
        imgs, labels = imgs.to(device), labels.to(device)
        
        # Apply mixup
        inputs, targets_a, targets_b, lam = mixup_data(imgs, labels, alpha=mixup_alpha)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        
        progress.set_postfix(loss=loss.item())
    
    return running_loss / len(loader.dataset)

# Validation function
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        progress = tqdm(loader, desc="Validation")
        for imgs, labels in progress:
            imgs, labels = imgs.to(device), labels.to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = correct / total * 100
    return running_loss / len(loader.dataset), accuracy, all_preds, all_labels

# Training loop
best_acc = 0.0
early_stop_counter = 0

print("Starting training...")
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}/{epochs}")
    
    # Learning rate warmup
    if epoch <= warmup_epochs:
        lr = learning_rate * (epoch / warmup_epochs)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    
    # Train
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    history['train_loss'].append(train_loss)
    
    # Validate
    val_loss, val_acc, val_preds, val_labels = validate(model, test_loader, criterion, device)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    # Update learning rate
    if epoch > warmup_epochs:
        scheduler_cosine.step()
        scheduler_plateau.step(val_acc)
    
    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        early_stop_counter = 0
        torch.save(model.state_dict(), save_path)
        print(f"New best model saved with accuracy: {val_acc:.2f}%")
        
        # Print classification report for best model
        print("\nClassification Report:")
        print(classification_report(val_labels, val_preds, target_names=classes))
    else:
        early_stop_counter += 1
        print(f"No improvement for {early_stop_counter}/{patience} epochs")
    
    # Early stopping
    if early_stop_counter >= patience:
        print(f"Early stopping at epoch {epoch}")
        break

print(f"\nBest Validation Accuracy: {best_acc:.2f}%")

# Plot training history
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Loss History')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(history['val_acc'], 'g-', label='Val Accuracy')
plt.title('Accuracy History')
plt.xlabel('Epochs')
plt.legend()

plt.tight_layout()
plt.savefig(history_path)
plt.show()

# Load best model
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

# Enhanced prediction function
def predict(img_path, return_confidence=False):
    img = Image.open(img_path).convert('L')
    x = test_tf(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = F.softmax(logits, dim=1)
    idx = logits.argmax(dim=1).item()
    confidence = probs[0, idx].item()
    
    if return_confidence:
        return classes[idx], confidence
    return classes[idx]

Using device: cuda
Loading dataset for mean/std calculation...


Computing mean/std: 100%|██████████| 449/449 [00:31<00:00, 14.12it/s]


Computed gray mean: 0.5077, std: 0.2063
Creating datasets...
Classes: ['angry', 'disgusted', 'fearful', 'happy', 'neutral', 'sad', 'surprised'] Number of classes: 7
Class counts: [3995  436 4097 7215 4965 4830 3171]


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

Feature channels: [32, 64, 96, 128, 640]
MobileViTEmotion(
  (backbone): FeatureListNet(
    (stem): ConvNormAct(
      (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
    )
    (stages_0): Sequential(
      (0): BottleneckBlock(
        (conv1_1x1): ConvNormAct(
          (conv): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNormAct2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
        )
        (conv2_kxk): ConvNormAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (bn): BatchNormAct2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

...

Validation: 100%|██████████| 113/113 [00:05<00:00, 21.10it/s]


Train Loss: 1.6591 | Val Loss: 1.3142 | Val Acc: 65.12%
No improvement for 7/15 epochs

Epoch 89/100


Validation: 100%|██████████| 113/113 [00:05<00:00, 21.23it/s]


Train Loss: 1.6843 | Val Loss: 1.2592 | Val Acc: 68.18%
No improvement for 8/15 epochs

Epoch 90/100


Training:  58%|█████▊    | 262/449 [00:41<00:29,  6.38it/s, loss=2.37] 